# 生成数据集

In [21]:
import numpy as np
import torch
from torch.utils import data

def synthetic_data(w, b, num_examples):  #@save
    """生成y=Xw+b+噪声"""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)


# 读取数据集

In [22]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)


In [23]:
next(iter(data_iter))

[tensor([[ 1.3407,  1.8264],
         [ 0.8252, -0.1075],
         [-2.7580, -0.6455],
         [ 0.5279, -0.8532],
         [-1.3662,  1.6424],
         [-0.8348, -0.6766],
         [ 2.3840,  0.1101],
         [ 0.2971, -0.5684],
         [ 0.3665, -0.1730],
         [ 0.2870, -0.5667]]),
 tensor([[ 0.6715],
         [ 6.2070],
         [ 0.8928],
         [ 8.1424],
         [-4.1080],
         [ 4.8605],
         [ 8.5942],
         [ 6.7329],
         [ 5.5190],
         [ 6.7046]])]

# 定义模型

In [24]:
# nn: neural network
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

# 初始化参数

In [25]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

# 定义损失函数

In [26]:
loss = nn.MSELoss()

# 定义优化器

In [27]:
# trainer = torch.optim.SGD(net.parameters(), lr=0.03)
trainer = torch.optim.Adam(net.parameters(), lr=0.03)

# 训练

In [28]:
num_epochs = 10
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 4.677643
epoch 2, loss 0.259807
epoch 3, loss 0.005812
epoch 4, loss 0.000140
epoch 5, loss 0.000097
epoch 6, loss 0.000097
epoch 7, loss 0.000097
epoch 8, loss 0.000097
epoch 9, loss 0.000097
epoch 10, loss 0.000097


In [29]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0003,  0.0003])
b的估计误差： tensor([-0.0002])


In [30]:
w, b

(tensor([[ 2.0003, -3.4003]]), tensor([4.2002]))